In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from google.colab import drive
import librosa
import h5py
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorflow_version 2.x
import datetime, os
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with h5py.File('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/singlefile_singlenoise_6-1.hdf5', 'r') as hf:
    clear_train = hf.get('clear_timit_train')
    mixture_train = hf.get('mixture_timit_train')

    clear_train = np.array(clear_train)
    mixture_train = np.array(mixture_train)

In [0]:
noisyMean     = np.mean(mixture_train);
noisyStd      = np.std(mixture_train);
print(str(noisyMean))
print(str(noisyStd))
mixture_train = (mixture_train - noisyMean)/noisyStd;
cleanMean     = np.mean(clear_train);
cleanStd      = np.std(clear_train);
print(str(cleanMean))
print(str(cleanStd))
clear_train    = (clear_train - cleanMean)/cleanStd;

0.31977266552270367
0.6308110213572822
0.24716835
0.6214038


### Channels Last Approach
We will create the following structure: 
(296324,129, 8, 1)
(296324,129, 1, 1)


In [0]:
mixture_train_shaped = np.reshape(mixture_train, (mixture_train.shape[0],mixture_train.shape[1], mixture_train.shape[2], 1));
clear_train_shaped = np.reshape(clear_train, (clear_train.shape[0], clear_train.shape[1], 1, 1));
print(mixture_train_shaped.shape)
print(clear_train_shaped.shape)

(157, 129, 8, 1)
(157, 129, 1, 1)


In [0]:
from tensorflow.keras.layers import Conv2D, Input, LeakyReLU, Flatten, Dense, Reshape, Conv2DTranspose, BatchNormalization, Activation
from tensorflow.keras import Model, Sequential
# Can our autoencoder learn to recover the original digits. We will use a slightly different model with more filters per layer
inputs = tf.keras.layers.Input(shape=(129,8,1))
x = inputs
l2_strength = 0.0
# ----- 0 
x = tf.keras.layers.ZeroPadding2D(((4,4), (0,0)))(x)
x = Conv2D(filters=18, kernel_size=[9,8], strides=[1, 1], padding='valid', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

skip0 = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
                kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(skip0)
x = BatchNormalization()(x)

x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = tf.keras.layers.Conv2D(30, (5, 1), strides=(1, 100), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)

# ----- 1
x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

skip1 = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
                kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(skip1)
x = BatchNormalization()(x)

x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = tf.keras.layers.Conv2D(30, (5, 1), strides=(1, 100), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)

# ---- 2
for i in range(0,4):
  x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)

  x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
              kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
  x = Activation('relu')(x)
  x = BatchNormalization()(x)

# ---- end - 2
x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = x + skip1
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = tf.keras.layers.Conv2D(30, (5, 1), strides=(1, 100), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)

# ---- end - 1
x = Conv2D(filters=18, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = Conv2D(filters=30, kernel_size=[5,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = x + skip0
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = Conv2D(filters=8, kernel_size=[9,1], strides=[1, 1], padding='same', use_bias=False,
            kernel_regularizer=tf.keras.regularizers.l2(l2_strength))(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)

x = tf.keras.layers.Conv2D(30, (5, 1), strides=(1, 100), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)

# ---- end
x = tf.keras.layers.SpatialDropout2D(0.2)(x)
x = Conv2D(filters=1, kernel_size=[129,1], strides=[1, 1], padding='same')(x)

model = Model(inputs=inputs, outputs=x)

optimizer = tf.keras.optimizers.Adam(3e-4)
#optimizer = RAdam(total_steps=10000, warmup_proportion=0.1, min_lr=3e-4)

model.compile(optimizer=optimizer, loss='mse', 
              metrics=[tf.keras.metrics.RootMeanSquaredError('rmse')])
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 129, 8, 1)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 137, 8, 1)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 129, 1, 18)   1296        zero_padding2d[0][0]             
__________________________________________________________________________________________________
activation (Activation)         (None, 129, 1, 18)   0           conv2d[0][0]                     
_____________

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)

In [0]:
model.load_weights('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/denoiser_cnn_log_mel_generator.h5')

In [0]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True, baseline=0.14)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='batch')
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/singlefile_singlenoise_6_cnn_log_mel_generator.h5', 
                                                         monitor='val_loss', save_best_only=True)
model.fit(mixture_train_shaped, clear_train_shaped,
         validation_split=0.1,
         epochs=10000,
         batch_size=256, 
         shuffle=True,
         callbacks=[early_stopping_callback, tensorboard_callback, checkpoint_callback]
        )
model.save('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/singlefile_singlenoise_6-1.h5') 

In [0]:
model.fit(mixture_train_shaped, clear_train_shaped,
         validation_split=0.1,
         epochs=10000,
         batch_size=32, 
         shuffle=True,
         callbacks=[]
        )
model.save('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/singlefile_singlenoise_6-1.h5') 

### Loading an Existing Model to Continue Training

In [0]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


### Code Evaluation
Testing the success of the algorithm with the dataset.

In [0]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/nodelay_v4.h5')

# Show the model architecture
# model.summary()

## Testing the Model
Here I will be continuing to improve the model

In [7]:
noisyMean     = 0.015;
noisyStd      = 0.09
print(str(noisyMean))
print(str(noisyStd))

0.015
0.09


In [0]:
counter = 0
FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
Overlap = round(0.75 * FrameSize)
FFTSize = FrameSize  # FFT window size=FRAMESIZE
FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
NumSegments = 8
# noisyMean = 0.34470737
# noisyStd  = 0.8982022
cleanStd  = 0.62
cleanMean = 0.25

def convert_to_audio(outname,stft_magnitude,stft_phase,noisyAudio,fs=8000,):
    # Normal Case:
    # noisyMean = 0.22;
    # noisyStd  = 0.6217
    cleanMean = 0.25
    cleanStd  = 0.62


    # Converting the data back to human language
    FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
    Overlap   = round(0.75 * FrameSize)
    FFTSize   = FrameSize  # FFT window size=FRAMESIZE
    FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
    NumSegments = 8
    if noisyAudio == True:
      stft_magnitude_nonNormal = noisyStd*stft_magnitude+noisyMean
    else:
      stft_magnitude_nonNormal = cleanStd*stft_magnitude+cleanMean
    final_STFT = stft_magnitude_nonNormal*np.exp(1j*stft_phase)
    # Audio Converter
    converted_audio = librosa.istft(final_STFT, hop_length=Overlap, win_length=FFTSize,
                  window=scipy.signal.hamming(FrameSize,sym=False))
    wavfile.write(outname, rate=fs, data=converted_audio)

def stft_generator(filename,noisyAudio,fs=8000):
    FrameSize = 256  # 512 under 16KHz time=32ms(normally 20~30ms )
    Overlap = round(0.75 * FrameSize)
    FFTSize = FrameSize  # FFT window size=FRAMESIZE
    FrequencyBins = FrameSize // 2 + 1  # stft_matrix:np.ndarray [shape=(1 + n_fft/2, t)]
    NumSegments = 8

    # Reading the audio data
    audio_data, sr = librosa.load(filename, mono=False, sr=fs)  # if sr=None to read raw sample_rate

    if noisyAudio == False:
        stft_segments= librosa.stft(audio_data, n_fft=FrameSize, hop_length=Overlap, win_length=FFTSize,
                                              window=scipy.signal.hamming(FrameSize,sym=False))
        # Phase
        audio_phase = np.angle(stft_segments)
        # Magnitude matrix
        stft_segments = np.transpose(abs(stft_segments))
    else:
        noisySTFT = librosa.stft(audio_data, n_fft=FrameSize, hop_length=Overlap, win_length=FFTSize,
                                     window=scipy.signal.hamming(FrameSize,sym=False))
        audio_phase = np.angle(noisySTFT)
        # Magnitude matrix
        noisySTFT =np.abs(noisySTFT)
        new_noisy_STFT = np.concatenate((noisySTFT[:,0:NumSegments-1], noisySTFT), axis=1)
        stft_segments = np.zeros((new_noisy_STFT.shape[1] - NumSegments + 1, FrequencyBins, NumSegments))
        for index in range(0, new_noisy_STFT.shape[1] - NumSegments + 1):
            stft_segments[index,:,:] = new_noisy_STFT[:, index:index+NumSegments]

    return {'stft': stft_segments, 'phase': audio_phase}

In [10]:
# Creating the magnitude and phase spectrums of noisy signal
from scipy.io import wavfile
import scipy
stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/mixture_494_n2_3.wav',True)
clean_stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/4Channel/perfect494.wav',False)
absSTFT = stft_segments['stft']
noisyPhase = stft_segments['phase']
absSTFT_normalized = (absSTFT - noisyMean)/noisyStd
cleanPhase = clean_stft_segments['phase']
clean_stft_segments = clean_stft_segments['stft']
clean_absSTFT_normalized = (clean_stft_segments - cleanMean)/cleanStd
print(absSTFT.shape)
print(clean_stft_segments.shape)

(53, 129, 8)
(53, 129)


In [11]:
noisySTFT = np.reshape(absSTFT_normalized, (absSTFT_normalized.shape[0],absSTFT_normalized.shape[1], absSTFT_normalized.shape[2], 1));
cleanSTFT = np.reshape(clean_absSTFT_normalized, (clean_absSTFT_normalized.shape[0], clean_absSTFT_normalized.shape[1], 1, 1));
print(noisySTFT.shape)
print(cleanSTFT.shape)
cleanedup_audio = model.predict(noisySTFT)
print(cleanedup_audio.shape)

(53, 129, 8, 1)
(53, 129, 1, 1)
(53, 129, 1, 1)


In [12]:
print('\n# Evaluate on test data')
results = model.evaluate(noisySTFT, cleanSTFT,verbose=1)
print('test loss, test acc:', results)


# Evaluate on test data
2/2 [==============================] - 0s 4ms/step - loss: 3.5912
test loss, test acc: 3.5911755561828613


In [13]:
final_cleanedup_audio = np.reshape(cleanedup_audio, (noisySTFT.shape[0], noisySTFT.shape[1]));
print(final_cleanedup_audio.shape)
print(noisyPhase.shape)

final_cleanSTFT= np.reshape(cleanSTFT, (clean_stft_segments.shape[0], clean_stft_segments.shape[1]));
print(final_cleanSTFT.shape)
print(cleanPhase.shape)

(53, 129)
(129, 53)
(53, 129)
(129, 53)


### Write to the file
The code below converts the cleaned file to the drive.


In [0]:
convert_to_audio('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/mixture494_Cleaned_single.wav',
                 np.transpose(final_cleanedup_audio),noisyPhase,True)
# convert_to_audio('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/SingleNoise_SingleFile/timit7742_8K.wav',
#                  np.transpose(final_cleanSTFT),cleanPhase,False)

# PLAYGROUND FOR GROUND TRUTH

In [0]:
mse = (np.square(final_cleanedup_audio - final_cleanSTFT)).mean()
print(mse)
stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145_n1.wav',False)
absSTFT = stft_segments['stft']
noisyPhase = stft_segments['phase']
absSTFT_normalized = (absSTFT - noisyMean)/noisyStd
print(absSTFT_normalized.shape)
mse = (np.square(absSTFT_normalized - final_cleanSTFT)).mean()
print(mse)

In [0]:
# Creating the magnitude and phase spectrums of noisy signal
from scipy.io import wavfile
import scipy
stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145.wav',True)
clean_stft_segments = stft_generator('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145.wav',False)
absSTFT = stft_segments['stft']
noisyPhase = stft_segments['phase']
absSTFT_normalized = (absSTFT - cleanMean)/cleanStd
cleanPhase = clean_stft_segments['phase']
clean_stft_segments = clean_stft_segments['stft']
clean_absSTFT_normalized = (clean_stft_segments - cleanMean)/cleanStd
print(absSTFT.shape)
print(clean_stft_segments.shape)

(129, 8, 325)
(129, 325)


### Demo Code


In [0]:
for i in range(8):
  print(sum(np.ravel(absSTFT[:,i,:])-np.ravel(clean_stft_segments[:,:])))
cleanedup_audio = new_model.predict(noisySTFT)
print(absSTFT.shape)
print(clean_stft_segments.shape)


-5.521531762433824
-4.648348442810402
-3.690907558631352
-2.8582976624525145
-2.064867725527165
-1.2853483124349623
-0.4730873316739235
0.0
(129, 8, 325)
(129, 325)


In [0]:
noisySTFT = np.reshape(absSTFT_normalized, (absSTFT_normalized.shape[2],absSTFT_normalized.shape[0], absSTFT_normalized.shape[1], 1));
cleanSTFT = np.reshape(clean_absSTFT_normalized, (clean_absSTFT_normalized.shape[1], clean_absSTFT_normalized.shape[0], 1, 1));
noisySTFT = np.ravel(noisySTFT)
print(noisySTFT.shape)
noisySTFT = np.reshape(noisySTFT, (absSTFT_normalized.shape[0],absSTFT_normalized.shape[1], absSTFT_normalized.shape[2]));
print(noisySTFT.shape)
cleanSTFT = np.ravel(cleanSTFT)
print(cleanSTFT.shape)
cleanSTFT = np.reshape(cleanSTFT, (clean_absSTFT_normalized.shape[0], clean_absSTFT_normalized.shape[1]));
print(cleanSTFT.shape)

for i in range(8):
  print(sum(np.ravel(noisySTFT[:,i,:])-np.ravel(cleanSTFT[:,:])))
# cleanedup_audio = new_model.predict(noisySTFT)
# print(noisySTFT.shape)
# print(cleanSTFT.shape)
# print(cleanedup_audio.shape)
# print(noisyPhase.shape)

(335400,)
(129, 8, 325)
(41925,)
(129, 325)
-6.570148158180036
-5.531115976335604
-4.391822693334616
-3.4010702847510186
-2.456939516357768
-1.529361379701967
-0.5628228761593751
0.00012072501257150217


**Evaluation in numbers**


In [0]:
print('\n# Evaluate on test data')
results = new_model.evaluate(noisySTFT, cleanSTFT,verbose=1)
print('test loss, test acc:', results)


# Evaluate on test data
325/325 [==============================] - 0s 269us/sample - loss: 0.7077
test loss, test acc: 0.707741042226553


In [0]:
final_audio = np.transpose(np.squeeze(cleanSTFT))
print(final_audio[1:3,1:3])
print(clean_absSTFT_normalized[1:3,1:3])
print(final_audio.shape)
print(clean_absSTFT_normalized.shape)
print(cleanPhase.shape)

[[-0.28905934 -0.26405573]
 [-0.26985237 -0.3025639 ]]
[[-0.29344794 -0.2982164 ]
 [-0.3020195  -0.30220264]]
(129, 325)
(129, 325)
(129, 325)


In [0]:
convert_to_audio('/content/drive/My Drive/Colab Notebooks/ThesisData/Basic_NoDelay/TestData/timit2145furka_cleanedv2.wav',
                 clean_absSTFT_normalized,cleanPhase,False)